In [1]:
import os
import sys
import random
import math
import subprocess
from glob import glob
from collections import OrderedDict
import numpy as np
import cv2
from skimage import measure
import pandas as pd
from tqdm import tqdm_notebook as tqdm
from matplotlib import pyplot as plt
import scipy.ndimage as ndi
from PIL import Image
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn import metrics
from sklearn.linear_model import LinearRegression

import torch
from torch import nn
from torch.nn import functional as F
import torch.backends.cudnn as cudnn
from torch.optim import lr_scheduler
from torch.utils.data import DataLoader
import torchvision
from torchvision import datasets, models, transforms
from torch.optim.optimizer import Optimizer, required
import pretrainedmodels

In [23]:
os.makedirs('./data/images_test_288_scaled/', exist_ok=True)
os.makedirs('./data/images_288_scaled/', exist_ok=True)

In [2]:
train_dir = "./data/train_images/"
train_csv_dir = "./data/train.csv"
test_dir = "./data/test_images/"
test_csv_dir = "./data/test.csv"

In [3]:
def scale_radius(src, img_size, padding=False):
    x = src[src.shape[0] // 2, ...].sum(axis=1)
    r = (x > x.mean() / 10).sum() // 2
    yx = src.sum(axis=2)
    region_props = measure.regionprops((yx > yx.mean() / 10).astype('uint8'))
    yc, xc = np.round(region_props[0].centroid).astype('int')
    x1 = max(xc - r, 0)
    x2 = min(xc + r, src.shape[1] - 1)
    y1 = max(yc - r, 0)
    y2 = min(yc + r, src.shape[0] - 1)
    dst = src[y1:y2, x1:x2]
    dst = cv2.resize(dst, dsize=None, fx=img_size/(2*r), fy=img_size/(2*r))
    if padding:
        pad_x = (img_size - dst.shape[1]) // 2
        pad_y = (img_size - dst.shape[0]) // 2
        dst = np.pad(dst, ((pad_y, pad_y), (pad_x, pad_x), (0, 0)), 'constant')
    return dst

    
class Dataset(torch.utils.data.Dataset):
    def __init__(self, img_paths, labels, transform=None, img_size=288, save_img=True):
        self.img_paths = img_paths
        self.labels = labels
        self.transform = transform
        self.img_size = img_size
        self.save_img = save_img

    def __getitem__(self, index):
        img_path, label = self.img_paths[index], self.labels[index]
        
        if 'train' in img_path:
            if os.path.exists('./data/images_288_scaled/%s' %os.path.basename(img_path)):
                img = cv2.imread('./data/images_288_scaled/%s' %os.path.basename(img_path))

            else:
                img = cv2.imread(img_path)
                try:
                    img = scale_radius(img, img_size=self.img_size, padding=False)
                except Exception as e:
                    img = img
                if self.save_img:
                    cv2.imwrite('./data/images_288_scaled/%s' %os.path.basename(img_path), img)
        
        else:
            if os.path.exists('./data/images_test_288_scaled/%s' %os.path.basename(img_path)):
                img = cv2.imread('./data/images_test_288_scaled/%s' %os.path.basename(img_path))

            else:
                img = cv2.imread(img_path)
                try:
                    img = scale_radius(img, img_size=self.img_size, padding=False)
                except Exception as e:
                    img = img
                if self.save_img:
                    
                    cv2.imwrite('./data/images_test_288_scaled/%s' %os.path.basename(img_path), img)
                   
        
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = Image.fromarray(img)

        if self.transform is not None:
            img = self.transform(img)

        return img, label


    def __len__(self):
        return len(self.img_paths)


def get_model(model_name='resnet18', num_outputs=None, pretrained=True,
              freeze_bn=False, dropout_p=0, **kwargs):

    pretrained = 'imagenet' if pretrained else None
    model = pretrainedmodels.__dict__[model_name](num_classes=1000,
                                                  pretrained=pretrained)

    if 'dpn' in model_name:
        in_channels = model.last_linear.in_channels
        model.last_linear = nn.Conv2d(in_channels, num_outputs,
                                      kernel_size=1, bias=True)
    else:
        if 'resnet' in model_name:
            model.avgpool = nn.AdaptiveAvgPool2d(1)
        else:
            model.avg_pool = nn.AdaptiveAvgPool2d(1)
        in_features = model.last_linear.in_features
        if dropout_p == 0:
            model.last_linear = nn.Linear(in_features, num_outputs)
        else:
            model.last_linear = nn.Sequential(
                nn.Dropout(p=dropout_p),
                nn.Linear(in_features, num_outputs),
            )

    if freeze_bn:
        for m in model.modules():
            if isinstance(m, nn.BatchNorm2d):
                m.weight.requires_grad = False
                m.bias.requires_grad = False

    return model


class RAdam(Optimizer):
    def __init__(self, params, lr=1e-3, betas=(0.9, 0.999), eps=1e-8, weight_decay=0):
        defaults = dict(lr=lr, betas=betas, eps=eps, weight_decay=weight_decay)
        self.buffer = [[None, None, None] for ind in range(10)]
        super(RAdam, self).__init__(params, defaults)

    def __setstate__(self, state):
        super(RAdam, self).__setstate__(state)

    def step(self, closure=None):

        loss = None
        if closure is not None:
            loss = closure()

        for group in self.param_groups:

            for p in group['params']:
                if p.grad is None:
                    continue
                grad = p.grad.data.float()
                if grad.is_sparse:
                    raise RuntimeError('RAdam does not support sparse gradients')

                p_data_fp32 = p.data.float()

                state = self.state[p]

                if len(state) == 0:
                    state['step'] = 0
                    state['exp_avg'] = torch.zeros_like(p_data_fp32)
                    state['exp_avg_sq'] = torch.zeros_like(p_data_fp32)
                else:
                    state['exp_avg'] = state['exp_avg'].type_as(p_data_fp32)
                    state['exp_avg_sq'] = state['exp_avg_sq'].type_as(p_data_fp32)

                exp_avg, exp_avg_sq = state['exp_avg'], state['exp_avg_sq']
                beta1, beta2 = group['betas']

                exp_avg_sq.mul_(beta2).addcmul_(1 - beta2, grad, grad)
                exp_avg.mul_(beta1).add_(1 - beta1, grad)

                state['step'] += 1
                buffered = self.buffer[int(state['step'] % 10)]
                if state['step'] == buffered[0]:
                    N_sma, step_size = buffered[1], buffered[2]
                else:
                    buffered[0] = state['step']
                    beta2_t = beta2 ** state['step']
                    N_sma_max = 2 / (1 - beta2) - 1
                    N_sma = N_sma_max - 2 * state['step'] * beta2_t / (1 - beta2_t)
                    buffered[1] = N_sma

                    # more conservative since it's an approximated value
                    if N_sma >= 5:
                        step_size = group['lr'] * math.sqrt((1 - beta2_t) * (N_sma - 4) / (N_sma_max - 4) * (N_sma - 2) / N_sma * N_sma_max / (N_sma_max - 2)) / (1 - beta1 ** state['step'])
                    else:
                        step_size = group['lr'] / (1 - beta1 ** state['step'])
                    buffered[2] = step_size

                if group['weight_decay'] != 0:
                    p_data_fp32.add_(-group['weight_decay'] * group['lr'], p_data_fp32)

                # more conservative since it's an approximated value
                if N_sma >= 5:
                    denom = exp_avg_sq.sqrt().add_(group['eps'])
                    p_data_fp32.addcdiv_(-step_size, exp_avg, denom)
                else:
                    p_data_fp32.add_(-step_size, exp_avg)

                p.data.copy_(p_data_fp32)

        return loss


class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count


def quadratic_weighted_kappa(y_pred, y_true):
    if torch.is_tensor(y_pred):
        y_pred = y_pred.data.cpu().numpy()
    if torch.is_tensor(y_true):
        y_true = y_true.data.cpu().numpy()
    if y_pred.shape[1] == 1:
        y_pred = y_pred[:, 0]
    else:
        y_pred = np.argmax(y_pred, axis=1)
    return metrics.cohen_kappa_score(y_pred, y_true, weights='quadratic')


def train(train_loader, model, criterion, optimizer, epoch):
    losses = AverageMeter()
    scores = AverageMeter()

    model.train()

    for i, (input, target) in tqdm(enumerate(train_loader), total=len(train_loader)):
        input = input.cuda()
        target = target.cuda()

        output = model(input)
        loss = criterion(output.view(-1), target.float())

        # compute gradient and do optimizing step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        thrs = [0.5, 1.5, 2.5, 3.5]
        output[output < thrs[0]] = 0
        output[(output >= thrs[0]) & (output < thrs[1])] = 1
        output[(output >= thrs[1]) & (output < thrs[2])] = 2
        output[(output >= thrs[2]) & (output < thrs[3])] = 3
        output[output >= thrs[3]] = 4
        
        target[target < thrs[0]] = 0
        target[(target >= thrs[0]) & (target < thrs[1])] = 1
        target[(target >= thrs[1]) & (target < thrs[2])] = 2
        target[(target >= thrs[2]) & (target < thrs[3])] = 3
        target[target >= thrs[3]] = 4
        
        score = quadratic_weighted_kappa(output, target)

        losses.update(loss.item(), input.size(0))
        scores.update(score, input.size(0))

    return losses.avg, scores.avg


def validate(val_loader, model, criterion):
    losses = AverageMeter()
    scores = AverageMeter()

    # switch to evaluate mode
    model.eval()

    with torch.no_grad():
        for i, (input, target) in tqdm(enumerate(val_loader), total=len(val_loader)):
            input = input.cuda()
            target = target.cuda()

            output = model(input)
            loss = criterion(output.view(-1), target.float())
        
            thrs = [0.5, 1.5, 2.5, 3.5]
            output[output < thrs[0]] = 0
            output[(output >= thrs[0]) & (output < thrs[1])] = 1
            output[(output >= thrs[1]) & (output < thrs[2])] = 2
            output[(output >= thrs[2]) & (output < thrs[3])] = 3
            output[output >= thrs[3]] = 4
            score = quadratic_weighted_kappa(output, target)

            losses.update(loss.item(), input.size(0))
            scores.update(score, input.size(0))

    return losses.avg, scores.avg

# Pseudo Labeling

In [4]:
pseudo_probs = {}

aptos2019_df = pd.read_csv(train_csv_dir)
aptos2019_img_paths = train_dir + aptos2019_df['id_code'].values + '.png'
aptos2019_labels = aptos2019_df['diagnosis'].values

test_df = pd.read_csv(test_csv_dir)
test_img_paths = test_dir+"/" + test_df['id_code'].values + '.png'
print(test_img_paths[0])
test_labels = np.zeros(len(test_img_paths))

test_transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
])

test_set = Dataset(
    test_img_paths,
    test_labels,
    transform=test_transform)
test_loader = torch.utils.data.DataLoader(
    test_set,
    batch_size=32,
    shuffle=False,
    num_workers=2)

./data/test_images//0005cfc8afb6.png


## SE-ResNeXt50_32x4d

In [5]:
# create model
model = get_model(model_name='se_resnext50_32x4d',
                  num_outputs=1,
                  pretrained=False,
                  freeze_bn=True,
                  dropout_p=0)
model = model.cuda()
model.eval()

probs = []
for fold in range(5):
    print('Fold [%d/%d]' %(fold+1, 5))

    model.load_state_dict(torch.load('./saved/se_resnext50_32x4d_pre2015/model_%d.pth' % (fold+1)))

    probs_fold = []
    with torch.no_grad():
        for i, (input, _) in tqdm(enumerate(test_loader), total=len(test_loader)):
            input = input.cuda()
            output = model(input)

            probs_fold.extend(output.data.cpu().numpy()[:, 0])
    probs_fold = np.array(probs_fold)
    probs.append(probs_fold)

probs = np.mean(probs, axis=0)
pseudo_probs['se_resnext50_32x4d'] = probs

del model
torch.cuda.empty_cache()
!nvidia-smi

Fold [1/5]


/home/yuen/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:18: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`



Fold [2/5]



Fold [3/5]



Fold [4/5]



Fold [5/5]



Wed Dec 23 10:56:56 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.66       Driver Version: 450.66       CUDA Version: 11.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  GeForce GTX 108...  Off  | 00000000:02:00.0 Off |                  N/A |
| 51%   84C    P2   198W / 250W |   8858MiB / 11175MiB |     99%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+------

## SE-ResNeXt101_32x4d

In [6]:
# create model
model = get_model(model_name='se_resnext101_32x4d',
                  num_outputs=1,
                  pretrained=False,
                  freeze_bn=True,
                  dropout_p=0,
                  )
model = model.cuda()
model.eval()

probs = []
for fold in range(5):
    print('Fold [%d/%d]' %(fold+1, 5))

    model.load_state_dict(torch.load('./saved/se_resnext101_32x4d_pre2015/model_%d.pth' % (fold+1)))

    probs_fold = []
    with torch.no_grad():
        for i, (input, _) in tqdm(enumerate(test_loader), total=len(test_loader)):
            input = input.cuda()
            output = model(input)

            probs_fold.extend(output.data.cpu().numpy()[:, 0])
    probs_fold = np.array(probs_fold)
    probs.append(probs_fold)

probs = np.mean(probs, axis=0)
pseudo_probs['se_resnext101_32x4d'] = probs

del model
torch.cuda.empty_cache()
!nvidia-smi

Fold [1/5]


/home/yuen/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:19: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`



Fold [2/5]



Fold [3/5]



Fold [4/5]



Fold [5/5]



Wed Dec 23 10:59:05 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.66       Driver Version: 450.66       CUDA Version: 11.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  GeForce GTX 108...  Off  | 00000000:02:00.0 Off |                  N/A |
| 52%   83C    P2   181W / 250W |   8858MiB / 11175MiB |     97%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+------

## SENet154

In [7]:
# create model
model = get_model(model_name='senet154',
                  num_outputs=1,
                  pretrained=False,
                  freeze_bn=True,
                  dropout_p=0)
model = model.cuda()
model.eval()

probs = []
for fold in range(5):
    print('Fold [%d/%d]' %(fold+1, 5))

    model.load_state_dict(torch.load('./saved/senet154_pre2015/model_%d.pth' % (fold+1)))

    probs_fold = []
    with torch.no_grad():
        for i, (input, _) in tqdm(enumerate(test_loader), total=len(test_loader)):
            input = input.cuda()
            output = model(input)

            probs_fold.extend(output.data.cpu().numpy()[:, 0])
    probs_fold = np.array(probs_fold)
    probs.append(probs_fold)

probs = np.mean(probs, axis=0)
pseudo_probs['senet154'] = probs

del model
torch.cuda.empty_cache()
!nvidia-smi

Fold [1/5]


/home/yuen/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:18: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`



Fold [2/5]



Fold [3/5]



Fold [4/5]



Fold [5/5]



Wed Dec 23 11:03:34 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.66       Driver Version: 450.66       CUDA Version: 11.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  GeForce GTX 108...  Off  | 00000000:02:00.0 Off |                  N/A |
| 51%   83C    P2   130W / 250W |   8858MiB / 11175MiB |    100%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+------

# Train

In [8]:
l1_probs = {}

train_transform = []
train_transform = transforms.Compose([
    transforms.Resize((288, 288)),
    transforms.RandomAffine(
        degrees=(-180, 180),
        scale=(0.8889, 1.0),
        shear=(-36, 36),
    ),
    transforms.CenterCrop(256),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomVerticalFlip(p=0.5),
    transforms.ColorJitter(
        brightness=0,
        contrast=(0.9, 1.1),
        saturation=0,
        hue=0),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
])

val_transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
])

## SE-ResNeXt50_32x4d

### Train

In [9]:
aptos2019_df = pd.read_csv(train_csv_dir)
aptos2019_img_paths = train_dir + aptos2019_df['id_code'].values + '.png'
aptos2019_labels = aptos2019_df['diagnosis'].values

test_df = pd.read_csv(test_csv_dir)
test_img_paths = test_dir+"/" + test_df['id_code'].values + '.png'

test_labels = 0.4 * pseudo_probs['se_resnext50_32x4d'] + 0.3 * pseudo_probs['se_resnext101_32x4d'] + 0.3 * pseudo_probs['senet154']

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=41)
kf = KFold(n_splits=5, shuffle=True, random_state=41)
img_paths = []
labels = []
for (train_idx1, val_idx1), (train_idx2, val_idx2) in zip(skf.split(aptos2019_img_paths, aptos2019_labels), kf.split(test_img_paths)):
    img_paths.append((np.hstack((aptos2019_img_paths[train_idx1], test_img_paths[val_idx2])), aptos2019_img_paths[val_idx1]))
    labels.append((np.hstack((aptos2019_labels[train_idx1], test_labels[val_idx2])), aptos2019_labels[val_idx1]))

In [10]:
# create model
model = get_model(model_name='se_resnext50_32x4d',
                  num_outputs=1,
                  pretrained=False,
                  freeze_bn=True,
                  dropout_p=0,
                  )
model = model.cuda()

criterion = nn.MSELoss().cuda()

best_losses = []
best_scores = []
os.makedirs('./saved/se_resnext50_32x4d/', exist_ok=True)
for fold, ((train_img_paths, val_img_paths), (train_labels, val_labels)) in enumerate(zip(img_paths, labels)):
    print('Fold [%d/%d]' %(fold+1, len(img_paths)))

    # train
    train_set = Dataset(
        train_img_paths,
        train_labels,
        transform=train_transform,
        img_size=288,
        save_img=True)
    train_loader = torch.utils.data.DataLoader(
        train_set,
        batch_size=32,
        shuffle=True,
        num_workers=2)

    val_set = Dataset(
        val_img_paths,
        val_labels,
        transform=val_transform,
        save_img=True)
    val_loader = torch.utils.data.DataLoader(
        val_set,
        batch_size=32,
        shuffle=False,
        num_workers=2)

    model.load_state_dict(torch.load('./saved/se_resnext50_32x4d_pre2015/model_%d.pth' % (fold+1)))

    optimizer = RAdam(
        filter(lambda p: p.requires_grad, model.parameters()), lr=1e-3)

    scheduler = lr_scheduler.CosineAnnealingLR(
        optimizer, T_max=10, eta_min=1e-5)
    
    os.makedirs('se_resnext50_32x4d', exist_ok=True)

    best_loss = float('inf')
    best_score = 0
    for epoch in range(10):
        print('Epoch [%d/%d]' % (epoch + 1, 10))

        # train for one epoch
        train_loss, train_score = train(
            train_loader, model, criterion, optimizer, epoch)
        # evaluate on validation set
        val_loss, val_score = validate(val_loader, model, criterion)

        scheduler.step()

        print('loss %.4f - score %.4f - val_loss %.4f - val_score %.4f'
              % (train_loss, train_score, val_loss, val_score))

        if val_loss < best_loss:
            torch.save(model.state_dict(), './saved/se_resnext50_32x4d/model_%d.pth' %(fold+1))
            best_loss = val_loss
            best_score = val_score
            print("=> saved best model")

    print('val_loss:  %f' % best_loss)
    print('val_score: %f' % best_score)
    
del model
torch.cuda.empty_cache()
!nvidia-smi

Fold [1/5]
Epoch [1/10]


/home/yuen/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:207: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


/home/yuen/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:135: UserWarning: This overload of addcmul_ is deprecated:
	addcmul_(Number value, Tensor tensor1, Tensor tensor2)
Consider using one of the following signatures instead:
	addcmul_(Tensor tensor1, Tensor tensor2, *, Number value) (Triggered internally at  /pytorch/torch/csrc/utils/python_arg_parser.cpp:882.)


/home/yuen/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:248: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`



loss 0.2949 - score 0.8862 - val_loss 0.3078 - val_score 0.8785
=> saved best model
Epoch [2/10]


...


loss 0.1863 - score 0.9230 - val_loss 0.2690 - val_score 0.8981
Epoch [7/10]



loss 0.1652 - score 0.9331 - val_loss 0.2452 - val_score 0.8995
=> saved best model
Epoch [8/10]



loss 0.1414 - score 0.9420 - val_loss 0.2529 - val_score 0.9118
Epoch [9/10]



loss 0.1220 - score 0.9507 - val_loss 0.2340 - val_score 0.9048
=> saved best model
Epoch [10/10]



loss 0.1069 - score 0.9533 - val_loss 0.2331 - val_score 0.9062
=> saved best model
val_loss:  0.233087
val_score: 0.906222
Wed Dec 23 12:27:52 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.66       Driver Version: 450.66       CUDA Version: 11.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  GeForce GTX 108...  Off  | 00000000:02:00.0 Off |                  N/A |
| 52%   82C    P2    97W / 250W |   1413MiB / 11175MiB |     63%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------

## SE-ResNeXt101_32x4d

### Train

In [12]:
aptos2019_df = pd.read_csv(train_csv_dir)
aptos2019_img_paths = train_dir + aptos2019_df['id_code'].values + '.png'
aptos2019_labels = aptos2019_df['diagnosis'].values

test_df = pd.read_csv(test_csv_dir)
test_img_paths = test_dir+"/" + test_df['id_code'].values + '.png'

test_labels = 0.4 * pseudo_probs['se_resnext50_32x4d'] + 0.3 * pseudo_probs['se_resnext101_32x4d'] + 0.3 * pseudo_probs['senet154']

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=41)
kf = KFold(n_splits=5, shuffle=True, random_state=42)
img_paths = []
labels = []
for (train_idx1, val_idx1), (train_idx2, val_idx2) in zip(skf.split(aptos2019_img_paths, aptos2019_labels), kf.split(test_img_paths)):
    img_paths.append((np.hstack((aptos2019_img_paths[train_idx1], test_img_paths[val_idx2])), aptos2019_img_paths[val_idx1]))
    labels.append((np.hstack((aptos2019_labels[train_idx1], test_labels[val_idx2])), aptos2019_labels[val_idx1]))

In [16]:
# create model
model = get_model(model_name='se_resnext101_32x4d',
                  num_outputs=1,
                  pretrained=False,
                  freeze_bn=True,
                  dropout_p=0)
model = model.cuda()

criterion = nn.MSELoss().cuda()

best_losses = []
best_scores = []
os.makedirs('./saved/se_resnext101_32x4d/', exist_ok=True)
for fold, ((train_img_paths, val_img_paths), (train_labels, val_labels)) in enumerate(zip(img_paths, labels)):
    print('Fold [%d/%d]' %(fold+1, len(img_paths)))

    # train
    train_set = Dataset(
        train_img_paths,
        train_labels,
        transform=train_transform,
        img_size=288,
        save_img=True)

    train_loader = torch.utils.data.DataLoader(
        train_set,
        batch_size=32,
        shuffle=True,
        num_workers=2)

    val_set = Dataset(
        val_img_paths,
        val_labels,
        transform=val_transform,
        save_img=True)
    val_loader = torch.utils.data.DataLoader(
        val_set,
        batch_size=32,
        shuffle=False,
        num_workers=2)

    model.load_state_dict(torch.load('./saved/se_resnext101_32x4d_pre2015/model_%d.pth' % (fold+1)))

    optimizer = RAdam(
        filter(lambda p: p.requires_grad, model.parameters()), lr=1e-3)

    scheduler = lr_scheduler.CosineAnnealingLR(
        optimizer, T_max=10, eta_min=1e-5)
    
    os.makedirs('se_resnext101_32x4d', exist_ok=True)

    best_loss = float('inf')
    best_score = 0
    for epoch in range(10):
        print('Epoch [%d/%d]' % (epoch + 1, 10))

        # train for one epoch
        train_loss, train_score = train(
            train_loader, model, criterion, optimizer, epoch)
        # evaluate on validation set
        val_loss, val_score = validate(val_loader, model, criterion)

        scheduler.step()

        print('loss %.4f - score %.4f - val_loss %.4f - val_score %.4f'
              % (train_loss, train_score, val_loss, val_score))

        if val_loss < best_loss:
            torch.save(model.state_dict(), 'se_resnext101_32x4d/model_%d.pth' %(fold+1))
            best_loss = val_loss
            best_score = val_score
            print("=> saved best model")

    print('val_loss:  %f' % best_loss)
    print('val_score: %f' % best_score)
    
del model
torch.cuda.empty_cache()
!nvidia-smi

Fold [1/5]
Epoch [1/10]


/home/yuen/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:207: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


/home/yuen/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:248: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`



loss 0.2716 - score 0.9006 - val_loss 0.2807 - val_score 0.9038
=> saved best model
Epoch [2/10]



loss 0.2709 - score 0.8964 - val_loss 0.3137 - val_score 0.9018
Epoch [3/10]


...


loss 0.1822 - score 0.9232 - val_loss 0.2655 - val_score 0.8953
=> saved best model
Epoch [7/10]



loss 0.1704 - score 0.9327 - val_loss 0.2222 - val_score 0.9247
=> saved best model
Epoch [8/10]



loss 0.1430 - score 0.9436 - val_loss 0.2254 - val_score 0.9084
Epoch [9/10]



loss 0.1062 - score 0.9549 - val_loss 0.2232 - val_score 0.9111
Epoch [10/10]



loss 0.0987 - score 0.9582 - val_loss 0.2198 - val_score 0.9108
=> saved best model
val_loss:  0.219810
val_score: 0.910791
Wed Dec 23 13:35:37 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.66       Driver Version: 450.66       CUDA Version: 11.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  GeForce GTX 108...  Off  | 00000000:02:00.0 Off |                  N/A |
| 52%   82C    P2    97W / 250W |   1850MiB / 11175MiB |     54%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------

In [18]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [19]:
device

device(type='cuda', index=0)